In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("coursea_data.csv", index_col=0)
def convert_enrolled(val):
    if isinstance(val, str) and "k" in val.lower():
        return float(val.lower().strip().replace("k", "")) * 1000
    elif isinstance(val, str) and "m" in val.lower():
        return float(val.lower().strip().replace("m", "")) * 1000000
    else:
        return float(val)

df.course_students_enrolled = df.course_students_enrolled.apply(convert_k)

df.index = df.index.sort_values()

# Data visualization

## Duplicates

In [93]:
df[df[["course_title", "course_organization"]].duplicated(keep=False)]

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
224,Developing Your Musicianship,Berklee College of Music,COURSE,4.8,Mixed,41000.0
225,Developing Your Musicianship,Berklee College of Music,SPECIALIZATION,4.8,Beginner,54000.0


## Null values

In [94]:
df.isnull().sum()

course_title                0
course_organization         0
course_Certificate_type     0
course_rating               0
course_difficulty           0
course_students_enrolled    0
dtype: int64

## Rating and students enrolled outliers

### A total of 154 organizations, offering 888 diferent courses
- 17 courses falls to low outliers by rating, mean of 4.05 out of 5
- 78 courses with high outliers by students enrolled, mean of close to 460,000.
- None of these outliers intertwines

In [95]:
print("Total unique courses:", df.course_title.str.strip().nunique())
print("Total unique oragnizations:", df.course_organization.str.strip().nunique())

Total unique courses: 888
Total unique oragnizations: 154


In [96]:
num_csv = df.select_dtypes(include=[float, int])
descr = num_csv.describe()
q1 = descr.loc["25%"]
q3 = descr.loc["75%"]
iqr = q3-q1
lower = q1-1.5*iqr
upper = q3+1.5*iqr
outl_low = num_csv[(num_csv < lower)]
outl_high = num_csv[(num_csv > upper)]
outl_low.notna().sum()
outl_high.notna().sum()

low = out_low[out_low.notna()]
print(low.course_rating.dropna(how = "all").value_counts().sum())
print(low.mean().round(2))

high = out_high[out_high.notna()]
print(high.course_students_enrolled.dropna(how = "all").value_counts().sum())
print(high.mean().round())

17
course_rating               4.05
course_students_enrolled     NaN
dtype: float64
78
course_rating                    NaN
course_students_enrolled    459615.0
dtype: float64
